In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import missingno
import re
from transformers import pipeline, AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

2025-02-15 19:56:18.858026: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739645779.006877   10010 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739645779.069939   10010 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-15 19:56:19.613575: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
pd.options.display.float_format = '{:.2f}'.format

In [74]:
barcelona = pd.read_csv('/TFM Mercado Inmobiliario - Kschool/Metodología de Modelado/9. Código y datasets/Barcelona/Datasets/result_barcelona_districs_zip.csv', sep = ";")

In [4]:
barcelona_reduce = barcelona.copy()

In [5]:
barcelona_reduce['log_price'] = np.log(barcelona_reduce['price'])

In [ ]:
# Creamos categorias de precios segun codigo postal y el logaritmo del precio

labels = ['Muy Bajo', 'Bajo', 'Medio', 'Alto', 'Muy Alto']


avg_log_price_by_zip = barcelona_reduce.groupby('zip_code')['log_price'].mean()

price_categories = pd.qcut(avg_log_price_by_zip, q=5, labels=labels)

barcelona_reduce['price_category_by_zip'] = barcelona_reduce['zip_code'].map(price_categories)


print(barcelona_reduce[['zip_code', 'log_price', 'price_category_by_zip']].head())

   zip_code  log_price price_category_by_zip
0      8010      14.29              Muy Alto
1      8008      14.27              Muy Alto
2      8007      13.80              Muy Alto
3      8003      13.68                  Alto
4      8003      13.11                  Alto


In [7]:

labels = ['Muy Bajo', 'Bajo', 'Medio', 'Alto', 'Muy Alto']


price_categories = pd.qcut(barcelona_reduce['log_price'], q=5, labels=labels)


barcelona_reduce['price_category'] = price_categories

print(barcelona_reduce[['log_price', 'price_category']].head())

   log_price price_category
0      14.29       Muy Alto
1      14.27       Muy Alto
2      13.80       Muy Alto
3      13.68       Muy Alto
4      13.11           Alto


In [8]:
barcelona_reduce.columns

Index(['propertyCode', 'thumbnail', 'externalReference', 'numPhotos', 'floor',
       'price', 'propertyType', 'operation', 'size', 'exterior', 'rooms',
       'bathrooms', 'address', 'province', 'municipality', 'district',
       'country', 'neighborhood', 'locationId', 'latitude', 'longitude',
       'showAddress', 'url', 'description', 'hasVideo', 'status',
       'newDevelopment', 'favourite', 'newProperty', 'hasLift', 'priceByArea',
       'hasPlan', 'has3DTour', 'has360', 'hasStaging', 'urgentVisualHighlight',
       'visualHighlight', 'preferenceHighlight', 'topHighlight',
       'topNewDevelopment', 'topPlus', 'priceDropValue', 'dropDate',
       'priceDropPercentage', 'commercialName', 'contactName', 'userType',
       'phone1.phoneNumber', 'typology', 'subTypology', 'subtitle', 'title',
       'hasParkingSpace', 'isParkingSpaceIncludedInPrice', 'parkingSpacePrice',
       'hasSwimmingPool', 'hasTerrace', 'hasAirConditioning', 'hasBoxRoom',
       'hasGarden', 'zip_code', 'siz

In [75]:
barcelona_atributos = barcelona_reduce[['exterior','hasSwimmingPool','hasTerrace','hasAirConditioning','hasBoxRoom', 'hasGarden' ]]

In [77]:
# Reemplazar un valor específico en todo el DataFrame
barcelona_atributos.replace(True, 'True', inplace=True)
barcelona_atributos.replace(False, 'False', inplace=True)


/tmp/ipykernel_7236/2142779725.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  barcelona_atributos.replace(True, 'True', inplace=True)
/tmp/ipykernel_7236/2142779725.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  barcelona_atributos.replace(False, 'False', inplace=True)


In [ ]:
# Visualización de los valores únicos [False, No data, True] de cada columna
resumen = {col: barcelona_atributos[col].value_counts() for col in barcelona_atributos.columns}

resumen_df = pd.DataFrame(resumen).fillna(0).astype(int)

resumen_df

,exterior,hasSwimmingPool,hasTerrace,hasAirConditioning,hasBoxRoom,hasGarden
False,2437,4935,2967,3669,4617,4223
No data,0,3190,2608,2293,3630,3673
True,6481,793,3343,2956,671,1022


In [ ]:
# Aplicacamos lexaización y eliminación de stopwords

nltk.download('stopwords')
nltk.download('wordnet')


lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def clean_text(text):

    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'[^a-zA-Z0-9áéíóúÁÉÍÓÚñÑ\s]', '', text)

    text = text.lower()

    text = ' '.join([word for word in text.split() if word not in stop_words])

    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

    return text

barcelona['cleaned_description'] = barcelona['description'].apply(clean_text)

[nltk_data] Downloading package stopwords to nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# Hacemos una copia para evitar perder possibles datos
barcelona2 = barcelona.copy()

In [ ]:
# Mediante el algoritmo de Bert y con embeddings, se extraen las características de la descripción
# Actualizamos solo cuando el valor sea False o no data

ner_pipeline = pipeline("ner", model="google-bert/bert-base-multilingual-cased", device=-1)

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = AutoModel.from_pretrained("bert-base-multilingual-cased")

characteristics = {
    'floor': ['floor', 'piso'],
    'propertyType': ['property type', 'tipo de propiedad', 'tipo de inmueble'],
    'size': ['size', 'tamaño', 'superficie', 'metros cuadrados', 'm²'],
    'exterior': ['exterior', 'exterioridad'],
    'rooms': ['rooms', 'bedrooms', 'habitaciones', 'dormitorios'],
    'bathrooms': ['bathrooms', 'baños'],
    'hasLift': ['lift', 'elevator', 'ascensor', 'montacargas'],
    'hasParkingSpace': ['parking', 'garage', 'garaje', 'estacionamiento'],
    'hasSwimmingPool': ['pool', 'piscina', 'swimming pool'],
    'hasTerrace': ['terrace', 'terraza'],
    'hasAirConditioning': ['air conditioning', 'a/c', 'climatización', 'aire acondicionado'],
    'hasBoxRoom': ['box room', 'storage', 'almacén'],
    'hasGarden': ['garden', 'jardín']
}
def get_word_embedding(word):
    inputs = tokenizer(word, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    word_embedding = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return word_embedding


def find_most_similar_embeddings(embedding, feature_embeddings):
    similarities = cosine_similarity(embedding, feature_embeddings)
    most_similar_idx = np.argmax(similarities)
    return most_similar_idx, similarities[0][most_similar_idx]


def extract_characteristics_from_description(description):

    ner_results = ner_pipeline(description)

    extracted_features = {feature: False for feature in characteristics}

    description_lower = description.lower()

    for feature, words in characteristics.items():
        feature_embedding = np.vstack([get_word_embedding(word) for word in words])
        for word in words:

            if word in description_lower:
                extracted_features[feature] = True
                break
            else:
                word_embedding = get_word_embedding(word)
                most_similar_idx, similarity_score = find_most_similar_embeddings(word_embedding, feature_embedding)
                if similarity_score > 0.8:
                    extracted_features[feature] = True
                    break

    return extracted_features

def update_features_with_logging_optimized(barcelona2):

    modification_log = []

    for index, row in barcelona2.iterrows():
        description = row['cleaned_description']
        extracted_features = extract_characteristics_from_description(description)

        for feature, value in extracted_features.items():

            if pd.isna(row[feature]) or row[feature] == False:
                if value:
                    barcelona2.at[index, feature] = True
                    modification_log.append((index, feature, True))

    return barcelona2, modification_log

barcelona2, modification_log = update_features_with_logging_optimized(barcelona2)

for log_entry in modification_log:
    print(f"Línea {log_entry[0]}: Característica '{log_entry[1]}' actualizada a {log_entry[2]}")


barcelona2


KeyboardInterrupt: 

In [ ]:
characteristics = ['price', 'propertyType', 'operation', 'size', 'exterior', 'rooms',
       'bathrooms', 'hasParkingSpace','hasSwimmingPool', 'hasTerrace', 'hasAirConditioning', 'hasBoxRoom']

In [ ]:
barcelona2[['floor', 'propertyType', 'size', 'exterior', 'rooms', 'bathrooms',
    'hasLift', 'hasParkingSpace', 'hasSwimmingPool', 'hasTerrace',
    'hasAirConditioning', 'hasBoxRoom', 'hasGarden']].T

,4478,8144,3130,1992,2458,8848,222,1197,5041,212,...,6109,8716,7540,5552,1199,7968,4248,2876,2115,789
floor,3,9,No data,1,6,9,5,No data,1,1,...,1,bj,bj,3,1,2,4,11,2,5
propertyType,flat,flat,chalet,flat,penthouse,flat,flat,penthouse,flat,flat,...,flat,flat,duplex,flat,flat,flat,flat,flat,flat,flat
size,280.00,78.00,570.00,101.00,167.00,180.00,198.00,187.00,45.00,116.00,...,80.00,60.00,90.00,75.00,254.00,56.00,92.00,101.00,85.00,132.00
exterior,True,True,False,True,True,False,True,True,False,True,...,True,False,True,True,True,True,True,True,True,True
rooms,5,3,5,3,5,4,5,2,2,2,...,3,2,3,3,4,3,3,4,4,3
bathrooms,4,1,6,2,2,3,3,3,1,2,...,1,1,2,1,3,1,2,2,1,2
hasLift,True,True,True,True,True,True,True,True,False,True,...,True,True,True,False,True,False,True,True,True,True
hasParkingSpace,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
hasSwimmingPool,False,False,True,True,No data,True,No data,No data,False,No data,...,False,False,False,False,True,False,False,No data,No data,No data
hasTerrace,True,False,True,True,True,True,True,No data,No data,True,...,False,No data,True,False,True,False,False,No data,No data,No data


In [ ]:
barcelona2.drop(columns=[ 'cleaned_description'], inplace=True)

In [ ]:
barcelona2.to_csv('/TFM Mercado Inmobiliario - Kschool/Metodología de Modelado/9. Código y datasets/Barcelona/Datasets/result_barcelona_districs_zip_final.csv', sep = ";", index = False)